# 拉普拉斯方法定轨道六根数

## 1. 已知三次观测的观测坐标，定出某一时刻的位置矢量和速度矢量 ok
按照笔记的方法分别推出各个量

注意时间单位的选取!

## 2. 已知某一时刻的位置矢量和速度矢量，求轨道六根数

按照笔记方法,逐步求得：
a
n()
E()
e
M
i Omega w

In [97]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import datetime
import time

import DDS_CMpack as DDS

%matplotlib inline

In [117]:
# astronomy constants
R_earth = 6371e3 # m

M_earth = 5.965e24 # kg
G_graviation = 6.672e-11 # N·m^2 /kg^2 

au = 149597870e3 # m
r_station_earth = 0.999102 #*R_earth
r_earth_sun = 1*au

time_unit = 806.81163 #806.8116 # SI 

# miu_GM = 398600.5e-6 # km^3/SI^2

In [118]:
miu_GM = G_graviation*M_earth # 国际单位制

miu_GM

397984800000000.0

In [119]:
# input r0,v0,t0
r0 = np.array([-7121.76646542,-904.76855999,3187.35106048])*10**3 # unit= m
r0_norm = np.sqrt(sum([ i*i for i in r0])) # m

v0 = np.array([-1.21447382,-5.44512524,-4.36253157])*10**3 # unit= m/SI
v0_norm = np.sqrt(sum([ i*i for i in v0])) # m/SI

t0 = 121.0217445 # unit = SI from the first data we have
## t0 感觉应该要填别的

r0_norm/10**3,v0_norm/10**3

(7854.767375159511, 7.0820913039862115)

In [120]:
# get a 

a = 1/(2/r0_norm - v0_norm**2/miu_GM) # unit = m

a/10**3

7776.202221679585

In [130]:
# get n\E then a
n = np.sqrt(miu_GM/(a**3))

tan_E = (1 - r0_norm/a)*(a**2*n)/(np.dot(r0,v0))

e = np.sqrt( (1 - r0_norm/a)**2 + (np.dot(r0,v0) / (n*a**2))**2 )
# e = 0.01002

print(1 - r0_norm/a)
print(np.dot(r0,v0) / (n*a**2))

cos_E = (1 - r0_norm/a)/e
sin_E = (np.dot(r0,v0) / a**2*n)/e

e

-0.010103280655548108
-0.005916529745154699


0.011708185351713259

In [131]:
def arctan2(sinE,cosE):
    if sinE >=0 and cosE >=0: # I
        output = np.arcsin(sinE)
    elif sinE >=0 and cosE <=0: # II
        output = np.pi - np.arcsin(sinE)
    elif sinE <=0 and cosE >=0: # IV
        output = 2*np.pi + np.arcsin(sinE)
    else: # III
        output = np.pi - np.arcsin(sinE)
    return output


In [132]:
# E1 = np.arcsin(sin_E)
# E2 = np.arccos(cos_E)
# E3 = np.arctan(tan_E)

print(sin_E,cos_E,tan_E)

E = arctan2(sin_E,cos_E)

np.degrees(E)

-4.277020293051238e-07 -0.8629245567990342 1.707636248059451


180.00002450552117

In [124]:
# keplar 定 M
M = E - e*sin_E
M

3.1415930862994372

In [125]:
# 定 i Omega w

# method 1
P = (cos_E/r0_norm)*r0 - (sin_E/(a*n))*v0
Q = (sin_E/(r0_norm)*np.sqrt(1-e**2))*r0 + ((cos_E - e)/(a*n*np.sqrt(1-e**2)))*v0
R = np.cross(P,Q)

cos_i = R[2]
tan_Omega = -1*R[0]/R[1]

# get w ->[-90,+90]
tan_w = P[2]/Q[2]

# # method 2
h = np.cross(r0,v0)
h_norm = np.sqrt(sum([ i*i for i in h]))

# get cos_i, sin_i
cos_i_check = (h[2]/h_norm)
tan_i = np.sqrt(h[0]**2 + h[1]**2)/h[2]
sin_i = tan_i*cos_i_check


# get cos_\sin_Omega
sin_Omega = h[0]/(h_norm*sin_i)
cos_Omega = -h[1]/(h_norm*sin_i)

# finally get i Omega w

i = arctan2(sin_i,cos_i_check)

Omega = arctan2(sin_Omega,cos_Omega)

w = np.arctan(tan_w)

np.degrees(np.array([i,Omega,w]))

array([ 47.36168524,  31.3703739 , -33.28432527])

## 3. 由轨道6根数归算星历表

已知：
a, n(), E(), e, M, [i, Omega, w] = P,Q

t0 = 121.0217445 **# unit = SI from the first data we have**

M = M0 + n(t-t0)

M = E - esin(E)

In [126]:
# calculate keplar equation
# 设置初值t0 和我们需要归算的时间t

t = 0.124857*time_unit

Mt = M + n*(t - t0)

print(e*np.sin(Mt))
print(Mt)

0.0002184869384912764
3.1229305283589124


In [127]:
# 迭代 开普勒方程解 E
epsilon = 1e-20
key = 0
En0 = Mt
Enw = [En0]
Nwind = 0
while key == 0:
    Enw.append(Mt + e*np.sin(Enw[Nwind]))

    Nwind += 1
    print(Enw[Nwind])
    
    delta_Enw = np.abs(Enw[Nwind] - Enw[Nwind - 1])
    print(delta_Enw)
    print('------------')
    
    if delta_Enw < epsilon:
        key = 1;
        
Et = Enw[Nwind]

print('end:',Et)

3.1231490152974035
0.00021848693849113587
------------
3.1231464576520818
2.5576453217546202e-06
------------
3.123146487592374
2.994029202341153e-08
------------
3.1231464872418866
3.5048719482233537e-10
------------
3.1231464872459895
4.1029402098047285e-12
------------
3.1231464872459416
4.796163466380676e-14
------------
3.123146487245942
4.440892098500626e-16
------------
3.123146487245942
0.0
------------
end: 3.123146487245942


In [128]:
# 归算rt, vt

rt = a*(np.cos(Et) - e)*P + a*np.sqrt(1 - e**2)*Q
rt_norm = np.sqrt( np.dot(rt,rt) )

vt = -a**2*n/rt_norm*np.sin(Et)*P + a**2*n/rt_norm*np.sqrt(1 - e**2)*np.cos(Et)*Q

L_rt = rt/ np.sqrt( np.dot(rt,rt))

rt,vt

(array([-4999671.79776087,  4394831.24269462,  6901812.76786752]),
 array([ -945.00018468, -3872.17380401, -3056.31790065]))

In [129]:
# 计算赤经赤纬：
P_earth = rt
station_earth = np.array([-0.81000791, -0.25188507,  0.52786921])*R_earth

P_station = P_earth - station_earth

L_forecast = P_station/np.sqrt(np.dot(P_station,P_station))

L_forecast

array([0.02309183, 0.86110245, 0.50790682])

In [116]:
L = [-0.9152477093203553, -0.0501154444748697, -0.39976252051552885]
L = [-0.921558,0.385984,-0.041788]
L

[-0.921558, 0.385984, -0.041788]